In [ ]:
from typing import List, Tuple, Callable, Union


import pandas as pd
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
import rasterio


from rasterio.crs import CRS
from rasterio.transform import from_bounds
from shapely import force_2d
from shapely.geometry import box, Point, Polygon, MultiPolygon
from scipy.spatial import cKDTree
from sklearn.gaussian_process import GaussianProcessRegressor
from pykrige.rk import Krige
from pykrige.ok import OrdinaryKriging
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel, Kernel


from flood_density.plots import plot_gdf, city_bounds_and_density_plot, plot_kriging_results_with_basemap
from flood_density.preprocess import convert_kml_to_gdf, export_to_geojson,get_bounds_xy_min_max, extract_bounds_polygon, polygon_to_gdf, extract_city_data, coordinates_to_box, points_geocoordinates, convert_points_in_gdf, extract_city_bounds_from_df_to_gdf, gdf_to_geojson, clip_density_to_urban_area, prepare_coords, prepare_centroids, create_kriging_kernel, create_gpr_model, interpolate_grid, predict_grid, fit_gpr_model,convert_to_2d_grid

CRS_4326 = 4326


# Etapa 1 : Obtener datos .kml de la ciudad de La Plata.

In [ ]:
gdf_from_kml = convert_kml_to_gdf('laplata_cascourbano.kml')


In [ ]:
gdf_peligrosidad = export_to_geojson(gdf_from_kml, 'laplata_cascourbano.geojson')

In [ ]:
gdf_peligrosidad

In [ ]:
plot_peligrosidad = plot_gdf(gdf_peligrosidad)

# Etapa 2: Obtener las coordenadas de los puntos de la ciudad, formamos un polígono y luego, transformamos a un gdf.


In [ ]:
#¿Cómo obtengo bounds_dict? son coordenadas de la ciudad de La Plata Desde el archivo geojson

In [ ]:
#Extraemos las coordenadas geográficas de la ciudad 
city_bounds_coordinates= get_bounds_xy_min_max(gdf_peligrosidad)
city_bounds_coordinates

In [ ]:
# Convertimos las coordenadas de la ciudad a un box
boxx = coordinates_to_box(city_bounds_coordinates)

In [ ]:
#Usar el dataframe del polígono que forma las coordenadas de la LP
gdf_la_plata_from_polygon = gpd.GeoDataFrame({'geometry': [boxx]}, crs='EPSG:4326')

In [ ]:
# Convertir a EPSG 32721 (UTM zona 21S) --- lo uso para el plot de geotiff
#gdf_la_plata_from_polygon = gdf_la_plata_from_polygon.to_crs(epsg=32721)

# Etapa 3 : Obtener y transformar datos de densidad poblacional de la ciudad de La Plata.

Primero extraemos las filas que pertenecen a la ciudad de La Plata(sigue siendo un .csv), luego convertimos ese dataframe en un geodataframe.

In [ ]:
#Levantamos el dataset completo
df_arg = pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv") 

In [ ]:
# Extraemos las filas referidas a la ciudad de La Plata
df_lp_coordinates = extract_city_data(df_arg,'la_plata',city_bounds_coordinates)
df_lp_coordinates

In [ ]:
#Leemos los datos específicos de la ciudad de La Plata
df_lp_coordinates = pd.read_csv('la_plata_population_2020.csv')
df_lp_coordinates


In [ ]:
#Convertir dataframe de los puntos pertenecientes a la ciudad de La Plata en un geodataframe
gdf_lp_coordinates = extract_city_bounds_from_df_to_gdf(df_lp_coordinates, 'Y','X')

In [ ]:
# Extraemos las coordenadas de los puntos dentro de la ciudad de La Plata
casco_urbano = clip_density_to_urban_area(gdf_lp_coordinates,gdf_peligrosidad)

In [ ]:
# Reproyectar a EPSG 32721 (UTM zona 21S)
casco_urbano_utm = casco_urbano.to_crs(epsg=32721)
casco_urbano_utm

In [ ]:
# Calcular centroides en el CRS proyectado
centroides = casco_urbano_utm.geometry.centroid

In [ ]:
# Convertir centroides a un array de coordenadas
coords_centroides = np.array([[pt.x, pt.y] for pt in centroides])

In [ ]:
# Extraer densidades de casco_urbano_utm
values_density_lp = casco_urbano_utm['Z'].values

In [ ]:
#Extraer los límites del polígono de la ciudad de La Plata
bounds_lp = gdf_la_plata_from_polygon.total_bounds  # [minx, miny, maxx, maxy]
bounds_lp

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.scatter(coords_centroides[:, 0], coords_centroides[:, 1],
                c=values_density_lp, s=100,
                cmap='viridis', alpha=0.9,
                edgecolor='black', linewidth=1,
                marker='o')

plt.colorbar(sc, ax=ax, label='Densidad')
ax.set_title("Centroides (EPSG:32721)")
plt.show()

# Etapa 4 : Merge polígono y zona de densidad de la ciudad de La Plata en un plot para corroborar consistencia geográfica.

In [ ]:
city_bounds_and_density_plot(gdf_la_plata_from_polygon, coords_centroides, values_density_lp, bounds_lp)

# Etapa 5 :  Visualización de la densidad poblacional de La Plata usando kriging.

In [ ]:
casco_urbano_utm

## 1. Preparar los datos (coordenadas y centroides)

In [ ]:
centroids_lp = prepare_centroids(casco_urbano_utm, 'Z')

In [ ]:
coordinates_lp = prepare_coords(casco_urbano_utm)

## 2. Ajustar modelo kriging

In [ ]:
kernel = create_kriging_kernel(constant_value=1.0, length_scale=1000.0, noise_level=0.1,length_scale_bounds= (1e-2, 1e3), noise_level_bounds=(1e-10, 1e3))

## 3. Generar el modelo GaussianProcessRegressor


In [ ]:
gpr_model_kriging = create_gpr_model(kernel)

In [ ]:
gpr_kriging_fit = fit_gpr_model(gpr_model_kriging, coordinates_lp, centroids_lp)

## 4. Crear malla para interpolar


In [ ]:
grid_x, grid_y, grid_coords = interpolate_grid(bounds_lp, step=100)

## 5. Predecir en la malla



In [ ]:
predict_grid_lp = predict_grid(gpr_kriging_fit, (grid_x, grid_y, grid_coords))

## 6. Convertir resultados a grillas 2D


In [ ]:
grid_2d_lp = convert_to_2d_grid(predict_grid_lp, grid_x.shape)

In [ ]:
plot_kriging_results_with_basemap(casco_urbano_utm, coordinates_lp, centroids_lp, bounds_lp, grid_x, grid_y, gpr_kriging_fit, grid_2d_lp)

# Etapa 6 : Exportar plot a GeoTIFF

In [ ]:
# Exportar superficie interpolada
export_to_geotiff(grid_2d_lp, gdf_peligrosidad, gdf_la_plata_from_polygon, 'Densidad Poblacional Kriging', 32721)

## 1. Extraer grilla de datos

In [ ]:
#def extract_grid_from_tuple(grid_data: Tuple[np.ndarray]) -> np.ndarray:
#    
#    grid_array = grid_data[0]
#    
#    return grid_array.shape

In [ ]:
#grid_lp = extract_grid_from_tuple(grid_2d_lp)

## 2. Preparar límites geoespaciales y transformación para raster

In [ ]:
#def prepare_geospatial_bounds(gdf: gpd.GeoDataFrame, target_crs: int) -> tuple:
#    
#    # Obtener límites en coordenadas proyectadas
#    bounds_proj = gdf.total_bounds  # [xmin, ymin, xmax, ymax]
#
#    return bounds_proj

In [ ]:
#geo_bounds_lp = prepare_geospatial_bounds(gdf_la_plata_from_polygon, 32721)

In [ ]:
#type(geo_bounds_lp)

## 3. Escribir un array de numpy como archivo GeoTIFF

In [ ]:
#def write_geotiff(grid_array: np.ndarray, 
#                  bounds_proj:  Union[tuple, np.ndarray], 
#                  filename: str, 
#                  crs_epsg: int,
#                  description: str) -> str:
#    
#    bounds_array = bounds_proj[0] if len(bounds_proj) > 0 else bounds_proj
#
#    height, width = grid_array.shape
#    
#    # Calcular la transformación georreferenciada
#    transform = from_bounds(bounds_array[0], bounds_array[1], 
#                          bounds_array[2], bounds_array[3], 
#                          width, height)
#    
#    with rasterio.open(
#        filename,
#        'w',
#        driver='GTiff',
#        height=height,
#        width=width,
#        count=1,
#        dtype=grid_array.dtype,
#        crs=CRS.from_epsg(crs_epsg),
#        transform=transform,
#        compress='lzw'
#    ) as dst:
#        dst.write(grid_array, 1)
#        dst.set_band_description(1, description)
#    
#    return filename

In [ ]:
#write_geotiff(grid_lp,geo_bounds_lp,'kriging_densidad_poblacional.tif',32721,'Densidad Poblacional Kriging')

---------------------------------

In [ ]:
#Exportar una grilla de datos a formato GeoTIFF desde una tupla
#def export_to_geotiff(grid_data: Tuple[np.ndarray], 
#                     gdf: gpd.GeoDataFrame,
#                     gdf_2: gpd.GeoDataFrame,
#                     filename: str,
#                     crs_epsg: int = 32721):
#    
#    # Extraer el primer elemento (la grilla de datos)
#    grid_array = grid_data[0]
#
#    # Convertir las coordenadas del polígono a un objeto de caja
#    boxx = coordinates_to_box(city_bounds_coordinates)
#
#    #city_bounds_coordinates= get_bounds_xy_min_max(gdf)
#
#    #def get_bounds_xy_min_max(gdf: gpd.GeoDataFrame) -> Dict[str,float]:
#    ## Extraer límites
#    #minx, miny, maxx, maxy = gdf.total_bounds
#    #return {
#    #    "x_min": minx,
#    #    "y_min": miny,
#    #    "x_max": maxx,
#    #    "y_max": maxy
#    #}
#
#
#
#    # Usar el dataframe del polígono que forma las coordenadas de  La Plata (EOPSG:4326)
#    gdf_2= gpd.GeoDataFrame({'geometry': [boxx]}, crs='EPSG:4326')
#
#    # Transformar al CRS del plot (EPSG:32721)
#    gdf_2 = gdf_2.to_crs(epsg=32721)
#
#    # Obtener límites en coordenadas proyectadas
#    bounds_proj = gdf_2.total_bounds  # [xmin, ymin, xmax, ymax]
#    height, width = grid_array.shape
#    
#    # Calcula la transformación georreferenciada
#    transform = from_bounds(bounds_proj[0], bounds_proj[1], bounds_proj[2], bounds_proj[3], width, height)
#    
#    with rasterio.open(
#        filename,
#        'w',
#        driver='GTiff',
#        height=height,
#        width=width,
#        count=1,
#        dtype=grid_array.dtype,
#        crs=CRS.from_epsg(crs_epsg),
#        transform=transform,
#        compress='lzw'
#    ) as dst:
#        dst.write(grid_array, 1)
#        dst.set_band_description(1, 'Densidad Poblacional Kriging')

In [ ]:
# 5. Pipeline principal
#def run_kriging_pipeline(geodf, value_column, bounds, step=100):
#    coords, values = prepare_data(geodf, value_column)
#    model = fit_kriging(coords, values)
#    grid_xx, grid_yy, z = interpolate_grid(model, bounds, step)
#    plot_kriging_results(grid_xx, grid_yy, z, coords, values, bounds)